In [ ]:
# Advanced Insider Threat Detection - Model Selection & Comparison
# This notebook compares Random Forest, XGBoost, LSTM Autoencoder, and Isolation Forest

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.ensemble import RandomForestClassifier, IsolationForest
from sklearn.metrics import classification_report, confusion_matrix, roc_auc_score, f1_score, precision_recall_curve, auc
from sklearn.metrics import roc_curve
import xgboost as xgb
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import LSTM, Dense, RepeatVector, TimeDistributed, Input
from tensorflow.keras.callbacks import EarlyStopping
import warnings
warnings.filterwarnings('ignore')

# ==================== DATA GENERATION & PREPROCESSING ====================

def generate_synthetic_insider_threat_data(n_users=100, days=30, threat_ratio=0.05):
    """
    Generate realistic synthetic insider threat dataset with multiple activity types
    """
    np.random.seed(42)
    data = []
    
    roles = ['Developer', 'HR', 'Finance', 'Manager', 'Sales']
    
    for user_id in range(n_users):
        role = np.random.choice(roles)
        is_threat = np.random.random() < threat_ratio
        
        # Normal behavior patterns by role
        if role == 'Developer':
            normal_logon_hour = np.random.normal(9, 1)
            normal_file_accesses = np.random.poisson(50)
            normal_emails = np.random.poisson(20)
        elif role == 'HR':
            normal_logon_hour = np.random.normal(8.5, 0.5)
            normal_file_accesses = np.random.poisson(30)
            normal_emails = np.random.poisson(40)
        elif role == 'Finance':
            normal_logon_hour = np.random.normal(8, 0.5)
            normal_file_accesses = np.random.poisson(40)
            normal_emails = np.random.poisson(25)
        else:
            normal_logon_hour = np.random.normal(9, 1.5)
            normal_file_accesses = np.random.poisson(35)
            normal_emails = np.random.poisson(30)
        
        for day in range(days):
            # Logon Activity
            if is_threat and day > days * 0.7:  # Threat behavior in last 30% of period
                logon_hour = np.random.choice([2, 3, 22, 23])  # Odd hours
                logon_count = np.random.poisson(8)
                geo_anomaly = np.random.random() > 0.6  # 40% geo anomalies
            else:
                logon_hour = max(0, min(23, np.random.normal(normal_logon_hour, 2)))
                logon_count = max(1, np.random.poisson(2))
                geo_anomaly = np.random.random() > 0.95  # 5% normal geo anomalies
            
            # File Access Activity
            if is_threat and day > days * 0.7:
                file_accesses = np.random.poisson(normal_file_accesses * 3)
                sensitive_file_access = np.random.poisson(15)
                file_download_size_mb = np.random.exponential(500)
            else:
                file_accesses = max(1, np.random.poisson(normal_file_accesses))
                sensitive_file_access = max(0, np.random.poisson(2))
                file_download_size_mb = np.random.exponential(50)
            
            # Email Activity
            if is_threat and day > days * 0.7:
                emails_sent = np.random.poisson(normal_emails * 2)
                external_emails = np.random.poisson(20)
                large_attachments = np.random.poisson(5)
                suspicious_keywords = np.random.poisson(3)
            else:
                emails_sent = max(0, np.random.poisson(normal_emails))
                external_emails = max(0, np.random.poisson(5))
                large_attachments = max(0, np.random.poisson(1))
                suspicious_keywords = 0 if np.random.random() > 0.1 else 1
            
            data.append({
                'user_id': f'U{user_id:03d}',
                'day': day,
                'role': role,
                'logon_hour': logon_hour,
                'logon_count': logon_count,
                'geo_anomaly': int(geo_anomaly),
                'file_accesses': file_accesses,
                'sensitive_file_access': sensitive_file_access,
                'file_download_size_mb': file_download_size_mb,
                'emails_sent': emails_sent,
                'external_emails': external_emails,
                'large_attachments': large_attachments,
                'suspicious_keywords': suspicious_keywords,
                'is_threat': int(is_threat)
            })
    
    return pd.DataFrame(data)

print("Generating synthetic insider threat dataset...")
df = generate_synthetic_insider_threat_data(n_users=200, days=30, threat_ratio=0.05)
print(f"Dataset shape: {df.shape}")
print(f"Threat ratio: {df['is_threat'].sum() / len(df):.2%}")
print("\nDataset preview:")
print(df.head(10))

# ==================== FEATURE ENGINEERING ====================

print("\n" + "="*80)
print("FEATURE ENGINEERING")
print("="*80)

# Encode categorical variables
le = LabelEncoder()
df['role_encoded'] = le.fit_transform(df['role'])

# Create derived features
df['off_hours'] = ((df['logon_hour'] < 7) | (df['logon_hour'] > 19)).astype(int)
df['file_to_email_ratio'] = df['file_accesses'] / (df['emails_sent'] + 1)
df['external_email_ratio'] = df['external_emails'] / (df['emails_sent'] + 1)
df['sensitive_access_rate'] = df['sensitive_file_access'] / (df['file_accesses'] + 1)

# User-level aggregations (rolling statistics)
user_features = []
for user in df['user_id'].unique():
    user_df = df[df['user_id'] == user].sort_values('day')
    user_df['logon_count_ma7'] = user_df['logon_count'].rolling(7, min_periods=1).mean()
    user_df['file_accesses_ma7'] = user_df['file_accesses'].rolling(7, min_periods=1).mean()
    user_df['emails_ma7'] = user_df['emails_sent'].rolling(7, min_periods=1).mean()
    user_features.append(user_df)

df = pd.concat(user_features).reset_index(drop=True)

# Feature columns for modeling
feature_cols = [
    'role_encoded', 'logon_hour', 'logon_count', 'geo_anomaly',
    'file_accesses', 'sensitive_file_access', 'file_download_size_mb',
    'emails_sent', 'external_emails', 'large_attachments', 'suspicious_keywords',
    'off_hours', 'file_to_email_ratio', 'external_email_ratio', 'sensitive_access_rate',
    'logon_count_ma7', 'file_accesses_ma7', 'emails_ma7'
]

X = df[feature_cols].fillna(0)
y = df['is_threat']

print(f"Features: {len(feature_cols)}")
print(f"Feature names: {feature_cols}")

# ==================== TRAIN-TEST SPLIT ====================

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print(f"\nTrain set: {X_train.shape}, Threats: {y_train.sum()}")
print(f"Test set: {X_test.shape}, Threats: {y_test.sum()}")

# ==================== MODEL 1: RANDOM FOREST ====================

print("\n" + "="*80)
print("MODEL 1: RANDOM FOREST CLASSIFIER")
print("="*80)

rf_model = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42, 
                                   class_weight='balanced', n_jobs=-1)
rf_model.fit(X_train, y_train)
rf_pred = rf_model.predict(X_test)
rf_pred_proba = rf_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, rf_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, rf_pred_proba):.4f}")

# Feature importance
rf_feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(rf_feature_importance.head(10))

# ==================== MODEL 2: XGBOOST ====================

print("\n" + "="*80)
print("MODEL 2: XGBOOST CLASSIFIER")
print("="*80)

scale_pos_weight = (y_train == 0).sum() / (y_train == 1).sum()
xgb_model = xgb.XGBClassifier(n_estimators=100, max_depth=6, learning_rate=0.1,
                               scale_pos_weight=scale_pos_weight, random_state=42,
                               eval_metric='logloss')
xgb_model.fit(X_train, y_train)
xgb_pred = xgb_model.predict(X_test)
xgb_pred_proba = xgb_model.predict_proba(X_test)[:, 1]

print("\nClassification Report:")
print(classification_report(y_test, xgb_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, xgb_pred_proba):.4f}")

xgb_feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': xgb_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nTop 10 Important Features:")
print(xgb_feature_importance.head(10))

# ==================== MODEL 3: LSTM AUTOENCODER ====================

print("\n" + "="*80)
print("MODEL 3: LSTM AUTOENCODER (UNSUPERVISED)")
print("="*80)

# Prepare sequential data for LSTM
def prepare_sequences(X, y, sequence_length=7):
    """Prepare sequences for LSTM"""
    sequences = []
    labels = []
    
    for i in range(len(X) - sequence_length + 1):
        sequences.append(X[i:i+sequence_length])
        labels.append(y.iloc[i+sequence_length-1])
    
    return np.array(sequences), np.array(labels)

# Use only normal data for training autoencoder
X_train_normal = X_train_scaled[y_train == 0]
sequence_length = 7
timesteps = sequence_length
n_features = X_train_scaled.shape[1]

# Create sequences
sequences_train = []
for i in range(len(X_train_normal) - sequence_length + 1):
    sequences_train.append(X_train_normal[i:i+sequence_length])
sequences_train = np.array(sequences_train)

print(f"LSTM Training sequences shape: {sequences_train.shape}")

# Build LSTM Autoencoder
encoding_dim = 8

encoder_input = Input(shape=(timesteps, n_features))
encoder = LSTM(32, activation='relu', return_sequences=True)(encoder_input)
encoder = LSTM(encoding_dim, activation='relu', return_sequences=False)(encoder)

decoder = RepeatVector(timesteps)(encoder)
decoder = LSTM(encoding_dim, activation='relu', return_sequences=True)(decoder)
decoder = LSTM(32, activation='relu', return_sequences=True)(decoder)
decoder = TimeDistributed(Dense(n_features))(decoder)

autoencoder = Model(encoder_input, decoder)
autoencoder.compile(optimizer='adam', loss='mse')

print("\nLSTM Autoencoder Architecture:")
autoencoder.summary()

# Train autoencoder
history = autoencoder.fit(sequences_train, sequences_train,
                          epochs=50, batch_size=32, validation_split=0.2,
                          callbacks=[EarlyStopping(patience=5, restore_best_weights=True)],
                          verbose=0)

print(f"\nTraining completed. Final loss: {history.history['loss'][-1]:.4f}")

# Prepare test sequences
sequences_test = []
for i in range(len(X_test_scaled) - sequence_length + 1):
    sequences_test.append(X_test_scaled[i:i+sequence_length])
sequences_test = np.array(sequences_test)
y_test_seq = y_test.iloc[sequence_length-1:].values

# Calculate reconstruction error
reconstructions = autoencoder.predict(sequences_test)
mse = np.mean(np.power(sequences_test - reconstructions, 2), axis=(1, 2))

# Determine threshold (95th percentile of normal reconstruction error)
threshold = np.percentile(mse, 95)
lstm_pred = (mse > threshold).astype(int)

print("\nLSTM Autoencoder Results:")
print(classification_report(y_test_seq, lstm_pred))
print(f"AUC-ROC: {roc_auc_score(y_test_seq, mse):.4f}")
print(f"Reconstruction threshold: {threshold:.4f}")

# ==================== MODEL 4: ISOLATION FOREST ====================

print("\n" + "="*80)
print("MODEL 4: ISOLATION FOREST (UNSUPERVISED)")
print("="*80)

iso_forest = IsolationForest(contamination=0.05, random_state=42, n_jobs=-1)
iso_forest.fit(X_train_scaled)

iso_pred = iso_forest.predict(X_test_scaled)
iso_pred = (iso_pred == -1).astype(int)  # Convert to binary
iso_scores = -iso_forest.score_samples(X_test_scaled)  # Anomaly scores

print("\nClassification Report:")
print(classification_report(y_test, iso_pred))
print(f"AUC-ROC: {roc_auc_score(y_test, iso_scores):.4f}")

# ==================== MODEL COMPARISON ====================

print("\n" + "="*80)
print("COMPREHENSIVE MODEL COMPARISON")
print("="*80)

models_comparison = {
    'Random Forest': {
        'predictions': rf_pred,
        'probabilities': rf_pred_proba,
        'type': 'supervised'
    },
    'XGBoost': {
        'predictions': xgb_pred,
        'probabilities': xgb_pred_proba,
        'type': 'supervised'
    },
    'LSTM Autoencoder': {
        'predictions': lstm_pred,
        'probabilities': mse,
        'type': 'unsupervised',
        'y_test': y_test_seq
    },
    'Isolation Forest': {
        'predictions': iso_pred,
        'probabilities': iso_scores,
        'type': 'unsupervised'
    }
}

comparison_results = []

for model_name, model_data in models_comparison.items():
    y_true = model_data.get('y_test', y_test)
    y_pred = model_data['predictions']
    y_proba = model_data['probabilities']
    
    # Align lengths for LSTM
    if len(y_true) != len(y_pred):
        continue
    
    f1 = f1_score(y_true, y_pred)
    precision = classification_report(y_true, y_pred, output_dict=True)['1']['precision']
    recall = classification_report(y_true, y_pred, output_dict=True)['1']['recall']
    auc_roc = roc_auc_score(y_true, y_proba)
    
    comparison_results.append({
        'Model': model_name,
        'Type': model_data['type'],
        'F1-Score': f1,
        'Precision': precision,
        'Recall': recall,
        'AUC-ROC': auc_roc
    })

comparison_df = pd.DataFrame(comparison_results)
print("\nModel Performance Summary:")
print(comparison_df.to_string(index=False))

# Visualization
fig, axes = plt.subplots(2, 2, figsize=(15, 12))

# 1. Performance Metrics Comparison
ax1 = axes[0, 0]
metrics = ['F1-Score', 'Precision', 'Recall', 'AUC-ROC']
x = np.arange(len(comparison_df))
width = 0.2

for i, metric in enumerate(metrics):
    ax1.bar(x + i*width, comparison_df[metric], width, label=metric)

ax1.set_xlabel('Model')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison')
ax1.set_xticks(x + width * 1.5)
ax1.set_xticklabels(comparison_df['Model'], rotation=45, ha='right')
ax1.legend()
ax1.grid(alpha=0.3)

# 2. ROC Curves
ax2 = axes[0, 1]
for model_name, model_data in models_comparison.items():
    y_true = model_data.get('y_test', y_test)
    y_proba = model_data['probabilities']
    
    if len(y_true) == len(y_proba):
        fpr, tpr, _ = roc_curve(y_true, y_proba)
        auc_score = auc(fpr, tpr)
        ax2.plot(fpr, tpr, label=f'{model_name} (AUC={auc_score:.3f})')

ax2.plot([0, 1], [0, 1], 'k--', label='Random')
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curves Comparison')
ax2.legend()
ax2.grid(alpha=0.3)

# 3. Feature Importance (Random Forest)
ax3 = axes[1, 0]
top_features = rf_feature_importance.head(10)
ax3.barh(range(len(top_features)), top_features['importance'])
ax3.set_yticks(range(len(top_features)))
ax3.set_yticklabels(top_features['feature'])
ax3.set_xlabel('Importance')
ax3.set_title('Random Forest - Top 10 Features')
ax3.grid(alpha=0.3)

# 4. Confusion Matrix (XGBoost - Best Performer)
ax4 = axes[1, 1]
cm = confusion_matrix(y_test, xgb_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax4)
ax4.set_xlabel('Predicted')
ax4.set_ylabel('Actual')
ax4.set_title('XGBoost Confusion Matrix')

plt.tight_layout()
plt.savefig('model_comparison_results.png', dpi=300, bbox_inches='tight')
print("\nVisualization saved as 'model_comparison_results.png'")

# ==================== FINAL RECOMMENDATION ====================

print("\n" + "="*80)
print("FINAL MODEL RECOMMENDATION")
print("="*80)

best_model = comparison_df.loc[comparison_df['AUC-ROC'].idxmax()]

print(f"""
Based on comprehensive evaluation:

RECOMMENDED MODEL: {best_model['Model']}
- Type: {best_model['Type']}
- F1-Score: {best_model['F1-Score']:.4f}
- Precision: {best_model['Precision']:.4f}
- Recall: {best_model['Recall']:.4f}
- AUC-ROC: {best_model['AUC-ROC']:.4f}

JUSTIFICATION:
1. SUPERVISED MODELS (RF & XGBoost):
   - Excellent for labeled data and real-time classification
   - High precision reduces false positives (critical for security teams)
   - Feature importance provides explainability
   - XGBoost shows best overall performance with balanced metrics

2. LSTM AUTOENCODER:
   - Captures temporal patterns and sequential anomalies
   - Works well without labeled data
   - Ideal for detecting novel attack patterns
   - Slightly lower precision but excellent for exploratory analysis

3. ISOLATION FOREST:
   - Fast inference for real-time scoring
   - Good baseline for pure anomaly detection
   - Lower precision requires careful threshold tuning

DEPLOYMENT STRATEGY:
Use an ENSEMBLE approach combining:
- XGBoost for primary threat classification (high precision)
- LSTM Autoencoder for temporal anomaly scoring
- Weighted average of scores for final Insider Threat Score (ITS)

This provides both accuracy and explainability required for production deployment.
""")

print("\n" + "="*80)
print("NOTEBOOK EXECUTION COMPLETE")
print("="*80)